In [ ]:
# This function does what it's supposed to; however,
# it could easily be replaced with something like 
# annotations = clinical[['Histologic_Grade_FIGO', 'Diabetes]]
# Initially I was trying to get this to do the whole process
# interactively, so people would just input values and it would
# work, but I have since split off those parts to try and test them.
# They don't work inside this function (so far), and only some kind of
# worked outside this function. So I simplified. A LOT.

def returnSubsetDF(df):
    print('Here are your options')
    print(set(df))
    boundary_names = []
    print('Please input the columns you would like to binarize. Type "done" to finish.')
    boundary = ''
    while boundary != 'done':
        boundary = input()
        if boundary == 'done':
            break
        elif boundary not in set(df):
            print('Invalid input\nHere are your options:')
            print(set(df))
        else:
            boundary_names.append(boundary)
    new_df = pd.DataFrame()
    for boundary in boundary_names:
        new_df[boundary] = df[boundary]
    return(new_df)

smaller_df = returnSubsetDF(clinical)

In [ ]:
# While this is a great idea and all, I can't figure out how to do this correctly

def binarizeV2(df):
    print('The following columns have been chosen for binarization:\n')
    boundary_names = list(set(df))
    example_dict = {}
    print('Please provide a dictionary to binarize them in the following format:\n')
    for boundary in boundary_names:
          print(boundary)
          example_dict[boundary] = {'option1': ['values', 'that', 'correspond', 'with', 'option1'],
                                   'option2': ['values', 'that', 'correspond', 'with', 'option2'], 
                                    'NaN' : [np.nan]
                                   }
    print(example_dict)
    

def binarizeDictionary(df, boundary_dict):
    boundary_names = list(set(df))
    for boundary in boundary_names:
        for key, value in boundary_dict.items():
            new_col = df[boundary].where(cond=(df[boundary] not in value), other=key)
        new_df[boundary] = new_col
    return(new_df)

my_dictionary = {'Beginning_Stage' : ['IA', 'IB', 'II'], 
                 'Later_Stage' : ['IIIA', 'IIIB', 'IIIC1', 'IIIC2', 'IVB'], 
                 np.nan : [np.nan]
                }
binarizeDictionary(smaller_df, my_dictionary)                
#binarizeV2(smaller_df)